In [1]:
import numpy as np

def Lstsq(regressor, response):
    # use numpy.linalg.lstsq
    # modify residual and singular value, eliminate the effect caused by data size
    if regressor.shape[0] <= regressor.shape[1]:
        raise ValueError("regressor size error, datums <= variables")
    
    output = np.linalg.lstsq(regressor, response, rcond=-1)
    output[1] /= regressor.shape[0] - regressor.shape[1]
    output[3] /= np.sqrt(regressor.shape[0]-1)
    return output

class LinearRegression():
    def __init__(self, regressor, response, has_bias = False, regularizer = None):
        if (regressor.ndim != 2) or (regressor.size == 0):
            raise ValueError("regressor should be a non-empty numpy matrix")
        elif (response.ndim != 2) or (response.size == 0):
            raise ValueError("response should be a non-empty numpy matrix")
        elif len(regressor) != len(response):
            raise ValueError("len(regressor) != len(response)")
        
        self.has_bias = has_bias
        if self.has_bias:
            self.regressor = np.append(regressor, np.ones((len(regressor), 1)), axis=1)
        else:
            self.regressor = regressor
        
        self.response = response
        self.regularizer = regularizer
    
    def MostUselesscAnalysis(self):
        gramian = np.dot(self.regressor.T, self.regressor) + self.regularizer * np.identity(regressor.shape[1])
        